In [2]:
import nltk
from nltk.corpus import reuters
import pandas as pd

nltk.download('reuters')

files = reuters.fileids()

dados = []

for file_id in files:
    categorias = reuters.categories(file_id)
    texto = reuters.raw(file_id)

    dados.append({
        "file_id": file_id,
        "categorias": ", ".join(categorias),
        "texto": texto
    })

df = pd.DataFrame(dados)

df.to_csv("reuters.csv", index=False, encoding="utf-8")



[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\strik\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!


In [3]:
random_samples = df.sample(10, random_state=42)
random_samples.to_csv("random_reuters_samples.csv", index=False, encoding="utf-8")
print("10 random rows saved to random_reuters_samples.csv")

df = df.drop(random_samples.index)

df.to_csv("reuters.csv", index=False, encoding="utf-8")
print("Original dataset updated in reuters.csv with 10 rows removed.")

10 random rows saved to random_reuters_samples.csv
Original dataset updated in reuters.csv with 10 rows removed.


In [4]:
df_reuters = pd.read_csv('reuters.csv')
df_random_samples = pd.read_csv('random_reuters_samples.csv')

print(f"Number of rows in reuters.csv: {len(df_reuters)}")
print(f"Number of rows in random_reuters_samples.csv: {len(df_random_samples)}")

Number of rows in reuters.csv: 10778
Number of rows in random_reuters_samples.csv: 10
